In [0]:
from bs4 import BeautifulSoup, Comment
import requests
import re
import pandas as pd
import numpy as np
import time
from google.colab import files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls drive/My\ Drive

 2018_schedule.csv	        IoT_and_ML_2019   論文
 2018_schedule_old.csv.gsheet   map		  通訊錄20120504.gsheet
'Colab Notebooks'	       'Shan&Des'
 gamelogs		        履歷


In [0]:
basePage = "https://www.baseball-reference.com"
playerStatPage = "/players/gl.fcgi"

In [0]:
def monthToNum(x):
    return {
        'January'  : '01',
        'February' : '02',
        'March'    : '03',
        'April'    : '04',
        'May'      : '05',
        'June'     : '06',
        'July'     : '07',
        'August'   : '08',
        'September': '09',
        'October'  : '10',
        'November' : '11',
        'December' : '12'
    }.get(x, None)

def parseTimeString(str):
    x = str.split(', ')
    y= x[1].split()
    return x[2]+"-"+monthToNum(y[0])+"-"+y[1]

In [0]:
def parseGameBox(boxlink):
    outString = ""
    lineupDict = {"AWAY": [], "HOME": []}
    pitcherDict = {"AWAY": [], "HOME": []}

    req = requests.get(boxlink).content.decode('utf-8')                            
    soup = BeautifulSoup(req,'html.parser')
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for c in comments:
        outString += c

    with open("temp.txt", "w") as fp:
        fp.write(outString)

    with open("temp.txt", "r") as fp:
        soup = BeautifulSoup(fp)

    btable = soup.find_all('div', id=re.compile("^lineups_\d*"))
    for i in range(2):
        players = btable[i].find_all('a')
        print("#### {} Team Lineup ####".format(list(lineupDict.keys())[i]))
        for player in players:
            if player.parent.string == None:
                continue

            print(player.string, end=", ")

            playerID = player['href'].split('/')[-1].split('.')[0]
            lineupDict[list(lineupDict.keys())[i]].append(playerID)

            getAttr = "?id={}&t=b&year=2018".format(playerID)
            playerLink = basePage+playerStatPage+getAttr
        print("")

    ptable = soup.find_all('table', id=re.compile(".*pitching$"))
    print("#### Pitchers ####")
    for i in range(2):
        pitcher = ptable[i].find('tbody').find('th').find('a')
        playerID = pitcher['href'].split('/')[-1]
        playerID = re.sub('\.shtml$', '', playerID)
        pitcherDict[list(pitcherDict.keys())[i]] = playerID
        print(" {}: {}".format(list(pitcherDict.keys())[i], pitcher.string))

        getAttr = "?id={}&t=p&year=2018".format(playerID)
        playerLink = basePage+playerStatPage+getAttr
    print("")

    return lineupDict, pitcherDict

In [0]:
def parseSchedule(scheduleLink):
    gmaesDfColNames = ['Date', 'Away', 'Home', 'A_runs', 'H_runs', 'A_sp', 'H_sp',
                    'A_b1', 'A_b2', 'A_b3', 'A_b4', 'A_b5', 'A_b6', 'A_b7', 'A_b8', 'A_b9',
                    'H_b1', 'H_b2', 'H_b3', 'H_b4', 'H_b5', 'H_b6', 'H_b7', 'H_b8', 'H_b9']
    gmaes_df = pd.DataFrame(columns=gmaesDfColNames)

    req = requests.get(scheduleLink).content.decode('utf-8')                            
    soup = BeautifulSoup(req,'html.parser')
    regularSeason = soup.find('div', 'section_content', id="div_2360547450")
    gameDays = regularSeason.find_all('div')

    cnt = 1
    for day in gameDays:
        if day.find(string="(Spring)") is not None:
            continue;
        date = parseTimeString(day.h3.string)
        print(date)

        #if date == "2018-3-29":
        #    continue

        games = day.find_all('p', 'game')
        for game in games:
            rowList = []
            rowList.append(date)

            match = game.find_all('a')
            away = match[0].string
            home = match[1].string
            rowList.append(away)
            rowList.append(home)
            print(away, "@", home)

            scores = game.find_all(string=re.compile("\(\d+\)"))
            scores[0] = int(re.findall(r'\d+', scores[0])[0])
            scores[1] = int(re.findall(r'\d+', scores[1])[0])
            rowList.append(scores[0])
            rowList.append(scores[1])

            boxlink = basePage + match[2]['href']
            lineupDict, pitcherDict= parseGameBox(boxlink)
            
            rowList.append(pitcherDict['AWAY'])
            rowList.append(pitcherDict['HOME'])
            rowList += lineupDict['AWAY']
            rowList += lineupDict['HOME']

            gmaes_df.loc[cnt] = rowList
            cnt += 1

    return gmaes_df

In [8]:
gmaes_df = parseSchedule("https://www.baseball-reference.com/leagues/MLB/2018-schedule.shtml")

2018-03-29
Colorado Rockies @ Arizona D'Backs
#### AWAY Team Lineup ####
Charlie Blackmon, DJ LeMahieu, Nolan Arenado, Trevor Story, Carlos Gonzalez, Ian Desmond, Gerardo Parra, Chris Iannetta, Jon Gray, 
#### HOME Team Lineup ####
David Peralta, A.J. Pollock, Paul Goldschmidt, Jake Lamb, Ketel Marte, Alex Avila, Nick Ahmed, Jarrod Dyson, Patrick Corbin, 
#### Pitchers ####
 AWAY: Jon Gray
 HOME: Patrick Corbin

Philadelphia Phillies @ Atlanta Braves
#### AWAY Team Lineup ####
Cesar Hernandez, Carlos Santana, Nick Williams, Rhys Hoskins, Aaron Altherr, J.P. Crawford, Maikel Franco, Andrew Knapp, Aaron Nola, 
#### HOME Team Lineup ####
Ender Inciarte, Ozzie Albies, Freddie Freeman, Nick Markakis, Tyler Flowers, Preston Tucker, Dansby Swanson, Ryan Flaherty, Julio Teheran, 
#### Pitchers ####
 AWAY: Aaron Nola
 HOME: Julio Teheran

Minnesota Twins @ Baltimore Orioles
#### AWAY Team Lineup ####
Brian Dozier, Joe Mauer, Miguel Sano, Eddie Rosario, Logan Morrison, Eduardo Escobar, Max Keple

In [0]:
gmaes_df.to_csv("/content/drive/My Drive/2018_schedule.csv", index=False)

In [0]:
#!zip -r file.zip batting
#files.download("file.zip")
#files.download("2018_schedule.csv")